In [3]:
import numpy as np
import pandas as pd
import glob as gb 
import os

### Состояние склада на каждый день

Создание имени файла

In [4]:
def create_filenames(letter, number):
    c = chr(ord('0') + number)
    str_i = 'MS-' + letter + c + '-inventory.csv'
    str_sl = 'MS-' + letter + c + '-sell.csv'
    str_spl = 'MS-' + letter + c + '-supply.csv'
    str_d = 'MS-' + letter + c + '-daily.csv'
    str_st = 'MS-' + letter + c + '-stolen'
    return str_i, str_sl, str_spl, str_d, str_st

Количество магазинов в штате

In [5]:
def num_shop(state):
    
    if state == 'b' or state == 'm':
        max = 2
    elif state == 's':
        max = 5
    return max

Чтение файлов 

In [6]:
def get_files(state, directory):
    max = num_shop(state)
    files = {}
    path = {}
    shop = 0
    while shop < max:
        c = chr(ord('0') + shop + 1)
        path[shop] = gb.glob(directory + state + c + '*.csv')
        print(path[shop])
        files[shop] = {}
        for j in range(0, len(path[shop])):
            print("opening: ", path[shop][j])
            files[shop][j]= pd.read_csv(path[shop][j], index_col='date', parse_dates=True, dayfirst=True)
        shop = shop + 1
    return files, max

Подсчет общего количества проданного товара за день

In [7]:
def sell_upg(sell):
    
    upg_tmp = pd.DataFrame(columns=['sku_num'], index = sell.index)
    upg_tmp['sku_num'] = sell['sku_num'].apply(lambda x: 0 if x.find('ap') == 6 else 1)
    daily = pd.crosstab(upg_tmp.index, upg_tmp['sku_num'], margins = True)
    daily = daily.drop(['All'])
    daily_new = pd.DataFrame(columns=['apple', 'pen'], index = upg_tmp.index.drop_duplicates())
    daily_new['apple'] = daily[0].values
    daily_new['pen'] = daily[1].values
        
    return daily_new

In [8]:
INV = 0
SELL = 1
SPL = 2
DAILY = 3
STOLEN = 4

### Task 1

In [9]:
def daily_store(sell, supply):

    daily_new = pd.DataFrame(columns=['apple', 'pen'], index = sell.index)
    daily_new['apple'] = -sell['apple']
    daily_new['pen'] = -sell['pen']

    df = pd.concat([supply, daily_new]).sort_index()
    df = df.resample('D').sum()

    l = df.index.size
    df['apple_sum'] = df['apple'].rolling(str(l) + 'D').sum()
    df['pen_sum'] = df['pen'].rolling(str(l) + 'D').sum()
    df = df.drop(['apple', 'pen'], axis=1)
    df = df.rename(columns={'apple_sum': 'apple', 'pen_sum': 'pen'})  
    return df
    

### Task 2

In [10]:
def stolen(sell, supply, inventory):
    st = pd.DataFrame(columns=['apple', 'pen'], index = inventory.index)
    sell = sell.resample('M').sum()
    supply = supply.resample('M').sum()
    inv = pd.DataFrame(columns=['apple', 'pen'], index = inventory.index)
    inv['apple'] = inventory['apple'] - inventory['apple'] .shift(1)
    inv.loc[inv.index[0], 'apple'] = inventory.loc[inventory.index[0], 'apple']
    inv['pen'] = inventory['pen'] - inventory['pen'] .shift(1)
    inv.loc[inv.index[0], 'pen'] = inventory.loc[inventory.index[0], 'pen']
    st['apple'] = inv['apple'] + sell['apple'] - supply['apple']
    st['pen'] = inv['pen'] + sell['pen'] - supply['pen'] 
    return st.abs()

### Task 3

In [11]:
def year_table(sold, stolen):
    states = ['b', 'm', 's']
    res = {}
    t = {}
    for i in range(0,3):
        t[i] = pd.DataFrame(index = sold[i].index)
    for state in range(0, 3):
        t[state] = sold[state].rename(columns= {'apple': 'apple_sold', 'pen': 'pen_sold'})
        t[state].insert(2, 'apple_stolen', stolen[state]['apple'])
        t[state].insert(3, 'pen_stolen', stolen[state]['pen'])
        t[state].insert(0, 'state', states[state])
    res = pd.concat([t[0], t[1], t[2]])
    print(res)
    return res
    

In [37]:
def check_res():
    states = ['b', 'm', 's']
    for state in states:
        right_ansm, maxou = get_files(state, './output/*')
        our_ans, maxa = get_files(state, './answer/*') 
        for i in range (0, maxou):
            if (right_ansm[i] != our_ans[i]):
                print('ответы не сошлись')

In [12]:
files = {}
max =  {}
files[0],  max[0] = get_files('b', './input/*')
files[1],  max[1] = get_files('m', './input/*')
files[2],  max[2] = get_files('s', './input/*')

['./input\\MS-b1-inventory.csv', './input\\MS-b1-sell.csv', './input\\MS-b1-supply.csv']
opening:  ./input\MS-b1-inventory.csv
opening:  ./input\MS-b1-sell.csv
opening:  ./input\MS-b1-supply.csv
['./input\\MS-b2-inventory.csv', './input\\MS-b2-sell.csv', './input\\MS-b2-supply.csv']
opening:  ./input\MS-b2-inventory.csv
opening:  ./input\MS-b2-sell.csv
opening:  ./input\MS-b2-supply.csv
['./input\\MS-m1-inventory.csv', './input\\MS-m1-sell.csv', './input\\MS-m1-supply.csv']
opening:  ./input\MS-m1-inventory.csv
opening:  ./input\MS-m1-sell.csv
opening:  ./input\MS-m1-supply.csv
['./input\\MS-m2-inventory.csv', './input\\MS-m2-sell.csv', './input\\MS-m2-supply.csv']
opening:  ./input\MS-m2-inventory.csv
opening:  ./input\MS-m2-sell.csv
opening:  ./input\MS-m2-supply.csv
['./input\\MS-s1-inventory.csv', './input\\MS-s1-sell.csv', './input\\MS-s1-supply.csv']
opening:  ./input\MS-s1-inventory.csv
opening:  ./input\MS-s1-sell.csv
opening:  ./input\MS-s1-supply.csv
['./input\\MS-s2-inventor

In [14]:
sstolen = {}
sold = {}
for i in range(0,3):
    sold[i] = pd.DataFrame(columns=['apple', 'pen'], index = files[i][0][SELL].index)
    sstolen[i] = pd.DataFrame(columns=['apple', 'pen'], index=files[i][0][SELL].index)
j = 0
state = ['b', 'm', 's']
for i in range(0,3):
    shop = 0
    while shop < max[j]:
        files[j][shop][DAILY] = sell_upg(files[j][shop][SELL])
        
        dly = daily_store(files[j][shop][DAILY], files[j][shop][SPL])
        name = create_filenames(state[i], shop + 1)[DAILY]
        dly.to_csv('./answer/' + name, index=False)
        
        files[j][shop][STOLEN] = stolen(files[j][shop][DAILY], files[j][shop][SPL], files[j][shop][INV]) 
        name = create_filenames(state[i], shop + 1)[STOLEN]
        files[j][shop][STOLEN].to_csv('./answer/' + name, index=False)

        sold[j] = pd.concat([sold[j], files[j][shop][DAILY]]).sort_index()
        sstolen[j] = pd.concat([sstolen[j], files[j][shop][STOLEN]]).sort_index()
        sold[j] = sold[j].resample('12M').sum()
        sstolen[j] = sstolen[j].resample('12M').sum()
        shop = shop + 1
    j = j + 1;


daily
               apple     pen
date                       
2006-01-01  33271.0  2574.0
2006-01-02  31409.0  2431.0
2006-01-03  29529.0  2260.0
2006-01-04  27732.0  2107.0
2006-01-05  25790.0  1974.0
stolen
             apple   pen
date                   
2006-01-31   10.0  11.0
2006-02-28    6.0   6.0
2006-03-31    7.0   6.0
2006-04-30    6.0  14.0
2006-05-31    8.0   1.0
else  0   b   0
daily
               apple     pen
date                       
2006-01-01  33271.0  2574.0
2006-01-02  31409.0  2431.0
2006-01-03  29529.0  2260.0
2006-01-04  27732.0  2107.0
2006-01-05  25790.0  1974.0
stolen
             apple   pen
date                   
2006-01-31   10.0  11.0
2006-02-28    6.0   6.0
2006-03-31    7.0   6.0
2006-04-30    6.0  14.0
2006-05-31    8.0   1.0
else  0   b   1
daily
               apple    pen
date                      
2006-01-01  14244.0  767.0
2006-01-02  13395.0  722.0
2006-01-03  12475.0  689.0
2006-01-04  11601.0  634.0
2006-01-05  10775.0  586.0
stolen
       

In [15]:
year = year_table(sold, sstolen)
year.to_csv('./answer/year_table.csv', index=False)

           state  apple_sold  pen_sold  apple_stolen  pen_stolen
date                                                            
2006-01-31     b      115842      9066          20.0        22.0
2007-01-31     b     1362080    104792         152.0       182.0
2008-01-31     b     1363028    103870         124.0       144.0
2009-01-31     b     1369680    104082         158.0       152.0
2010-01-31     b     1365148    105078         164.0       190.0
2011-01-31     b     1362124    104984         142.0       188.0
2012-01-31     b     1365296    104096         174.0       206.0
2013-01-31     b     1370318    105232         166.0       176.0
2014-01-31     b     1365902    104736         150.0       174.0
2015-01-31     b     1365274    103600         176.0       186.0
2016-01-31     b     1247436     95044         142.0       134.0
2006-01-31     m       52748      2862          12.0        18.0
2007-01-31     m      620198     32262         130.0       148.0
2008-01-31     m      618